In [1]:
import os
os.environ['MKL_THREADING_LAYER'] = 'GNU'

In [2]:
!pip install pymc3

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pymc3 as pm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
C:\Users\nicol\AppData\Roaming\Python\Python311\site-packages\theano\scalar\basic.py:2412: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  self.ctor = getattr(np, o_type.dtype)


AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [4]:
X = pd.read_csv('XValuesDataSet.csv')
Y = pd.read_csv('YValuesDataSet.csv')

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.5)

In [6]:
# Normalizar variables de entrada
X_train = (X_train - X_train.mean()) / X_train.std()

In [7]:
media = X_train.mean()
desviacion = X_train.std()

In [8]:
import multiprocessing
print(f"Number of CPU cores: {multiprocessing.cpu_count()}")

Number of CPU cores: 2


In [9]:
os.environ['MKL_NUM_THREADS'] = '6'
os.environ['OPENBLAS_NUM_THREADS'] = '6'
os.environ['OMP_NUM_THREADS'] = '6'

In [10]:
import theano
theano.config.gcc.cxxflags = "-Wno-c++11-narrowing"
theano.config.mode = 'FAST_RUN'

In [11]:
with pm.Model() as model:
    # crea variables aleatorias normales para cada columna
    betas = pm.Normal('betas', mu=media, sd=desviacion, shape=X_train.shape[1])
    
    # Probabilidad de éxito
    p = pm.math.sigmoid(pm.math.dot(X_train, betas))
    # Variable objetivo
    Y_train = pm.Bernoulli('y', p=p, observed=Y)
    
    # Inferencia de parámetros
    trace = pm.sample(20, tune=20, cores=2)

/usr/local/lib/python3.9/dist-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


/usr/local/lib/python3.9/dist-packages/pymc3/sampling.py:659: UserWarning: The number of samples is too small to check convergence reliably.
  warnings.warn("The number of samples is too small to check convergence reliably.")
ERROR:pymc3:There was 1 divergence after tuning. Increase `target_accept` or reparameterize.


In [42]:
# Crear lista vacía para guardar predicciones
predictions = []

# Loop sobre los valores de X_test y Y_test
for i in range(len(X_test)):
    # Obtener valor de X_test e Y_test para esta iteración
    x = X_test.iloc[[i]]
    y = Y_test.iloc[[i]]

    # Calcular probabilidad de éxito con el modelo
    with model:
        p = pm.math.sigmoid(pm.math.dot(x, betas))

    result_str = str(p)
    result_char = result_str[-1]
    result_float = float(result_char)

    # Obtener predicción como 1 o 0 según la probabilidad
    prediction = 1 if result_float > 0.5 else 0

    # Añadir predicción a la lista
    predictions.append(prediction)

# Comparar predicciones con valores reales
correct = 0
for i in range(len(Y_test)):
    if predictions[i] == Y_test.iloc[i, 0]:
        correct += 1

# Calcular accuracy
accuracy = correct / len(Y_test)
print(f"Accuracy: {accuracy}")

Accuracy: 0.5198883461270063
